In [ ]:
import openai
import os
from dotenv import load_dotenv

load_dotenv('.env')


openai.api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
class ChatBot():
    
    def answer(self, user_content):
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a chatbot. Hold a conversation. Respond using markdown."},
                {"role": "user", "content": user_content},
            ],
        )
        return response["choices"][0]["message"]["content"]

In [ ]:
mes = 'what is your name?'
chat = ChatBot()
a = chat.answer(mes)


Test retrval with chroma

In [ ]:
from chatbot import ChatBot 

# Instantiate the chatbot and layout
chatbot = ChatBot()

In [ ]:
def parse_sources(sources):
    # Create an empty list to store results
    parsed_sources = []
    # Iterate over all sources
    for source in sources:
        # Get the metadata from the source
        metadata = source['metadata']
        # Get the source name, or use a default if not found
        source_name = metadata.get('source', 'Unknown Source')
        # Get the page number, or use a default if not found
        page_number = metadata.get('page', 'Unknown Page')
        # Create the string and add it to the results
        parsed_sources.append(f"Source: {source_name}, Page: {page_number}")
    # Return the results
    return parsed_sources


In [ ]:
def parse_page_content(page_content):
    lines = page_content.split('\n')
    parsed_content = "\n".join(line for line in lines if line.strip())  # remove empty lines
    return parsed_content


In [ ]:
chat = chatbot.chat_bot_response('what is the law on gardeing?', [])

In [ ]:
pc = chat['source_documents'][0].page_content

In [ ]:
parse_page_content(pc)

In [ ]:
query = "Does pruning of a native tree trigger a permit requirement under the Significant landscape overlay schedule 2"
chat = chatbot.chat_bot_response(query, [])

In [6]:
chat

{'question': 'Does pruning of a native tree trigger a permit requirement under the Significant landscape overlay schedule 2',
 'chat_history': [],
 'answer': 'Yes, pruning of a native tree does trigger a permit requirement under the Significant Landscape Overlay Schedule 2. This is stated in clause 42.03-2 of the Nillumbik Planning Scheme, which requires a permit to be obtained for the construction of a building or to carry out works, including the pruning or lopping of the trunk of a native tree, unless a schedule to the overlay specifically states that a permit is not required. Therefore, if the schedule to the overlay does not specifically state that a permit is not required for pruning of a native tree, then a permit must be obtained.\n\nThe relevant clause is:\n\nVC224 Permit requirement\nA permit is required to:\n- Construct a building or to carry out works. This does not apply:\n- If a schedule to this overlay specifically states that a permit is not required.\n- To the conduct 

In [ ]:
query = "What are the laws relating to development of outdoor areas"
chat = chatbot.chat_bot_response(query, [])

test qdrant

In [2]:
from langchain.vectorstores import Qdrant
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFDirectoryLoader
from dotenv import load_dotenv
import os

In [3]:


load_dotenv()

# load the API key from the .env file
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# user_input = input("User: ")
# load the documents from the docs directory
loader = PyPDFDirectoryLoader("docs/")
docs = loader.load()

#split the documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(docs)

embedding = OpenAIEmbeddings()

In [4]:
qdrant = Qdrant.from_documents(
    docs,
    embedding,
    path="/tmp/local_qdrant",
    collection_name="my_documents",
)

In [7]:
query = "What are the laws relating to development of outdoor areas"

In [3]:
qdrant.similarity_search(query)

NameError: name 'qdrant' is not defined

In [8]:
url = "https://860d241f-fef7-48ab-ba8f-f81d6c2973b1.us-east-1-0.aws.cloud.qdrant.io"
api_key = "u-fxKHirhYZOqUkPTYUHa0vjF-eUhA-tPw_wB1rtZ74-Bg9WWQPgCQ"

In [9]:

qdrant = Qdrant.from_documents(
    docs,
    embedding,
    url=url,
    prefer_grpc=True,
    api_key=api_key,
    collection_name="my_documents",
)

In [12]:
del qdrant

NameError: name 'qdrant' is not defined

In [5]:

import qdrant_client

client = qdrant_client.QdrantClient(url=url, prefer_grpc=True, api_key=api_key)
qdrant = Qdrant(client=client, collection_name="my_documents", embeddings=embedding)


In [9]:
qdrant.similarity_search(query)


[Document(page_content='52.20-7.2 Communal open space\nAdevelopmentof10ormoredwellingsshouldprovideaminimumareaofcommunaloutdoor\nopenspaceof30squaremetres.\nIfadevelopmentcontains13ormoredwellings,thedevelopmentshouldalsoprovideanadditional\nminimumareaofcommunalopenspaceof2.5squaremetresperdwellingor220squaremetres,\nwhicheveristhelesser.Thisadditionalareamaybeindoorsoroutdoorsandconsistofmultiple\nseparateareasofcommunalopenspace.\nEachareaofcommunalopenspaceshouldbe:\nAccessibletoallresidents.\nAuseablesize,shapeanddimension.\nCapableofefficientmanagement.\nLocatedto:\n–Providepassivesurveillanceopportunities,whereappropriate.\n–Provideoutlookforasmanydwellingsaspracticable.\n–Avoidoverlookingintohabitableroomsandprivateopenspaceofnewdwellings.\n–Minimisenoiseimpactstonewandexistingdwellings.\nAnyareaofcommunaloutdooropenspaceshouldbelandscapedandincludecanopycoverand\ntrees.\n52.20-7.3 Solar access to communal outdoor open space\nThecommunaloutdooropenspaceshouldbelocatedonthenort

In [6]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.llm import LLMChain
from langchain import PromptTemplate
from langchain.vectorstores import Qdrant
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
import concurrent.futures
from dotenv import load_dotenv

import qdrant_client

import os
import openai

from dotenv import load_dotenv

load_dotenv('.env')
# openai.api_key = os.getenv('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
openai.api_key = os.getenv('OPENAI_API_KEY')
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")
QDRANT_API_URL = os.getenv("QDRANT_API_URL")

### For testing ###
# class ChatBot():
    
    # def chat_bot_response(self, user_content):
    #     response = openai.ChatCompletion.create(
    #         model="gpt-4",
    #         messages=[
    #             {"role": "system", "content": "You are a chatbot. Hold a conversation. Respond using markdown."},
    #             {"role": "user", "content": user_content},
    #         ],
    #     )
    #     return response["choices"][0]["message"]["content"]


class ChatBot:
    def __init__(self):
        # load vector db from disk
        # self.persist_directory = './db'
        # Get the absolute path to the directory of the current file
        # current_file_directory = os.path.dirname(os.path.abspath(__file__))

        # # Construct a path to the database
        # self.persist_directory = os.path.join(current_file_directory, 'db')

        self.embedding = OpenAIEmbeddings()

        # self.vectordb = Chroma(persist_directory=self.persist_directory, 
        #                     embedding_function=self.embedding)
        client = qdrant_client.QdrantClient(url=QDRANT_API_URL, prefer_grpc=True, api_key=QDRANT_API_KEY)
        self.vectordb = Qdrant(client=client, collection_name="my_documents", embeddings=self.embedding)

        # custom prompts
        self.CONDENSE_PROMPT = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.

        Chat History:
        {chat_history}
        Follow Up Input: {question}
        Standalone question:"""

        self.QA_PROMPT = """You will act as an AI Assistant expert in the Nillumbik Planning Scheme that I am having a conversation with. You have the expertise of a lawer with 10 years of experience in local town planning regulation for Nillumbik in Melbourne, Australia. You will provide answers and guidance from your extensive knowledge and will always provide answer when requested with exact reference and outline from the source documents. Additionally, you will ask follow-up questions to clarify the last response and provide more accurate and personalized answers. If the answer is not included in your knowledge base, you will say "Hmm, This may be the answer, but you should double check." and continue to answer normally using the context given. Your response should be detailed and use as much of the relevant documents as possible. Your goal is to provide the best possible guidance and support to help me with my queries and problems related to your expertise.  

        Example:      
        
        Question: Does pruning of a native tree trigger a permit requirement under the Significant landscape overlay schedule 2?
        
        Response:
        Yes, pruning of a native tree does trigger a permit requirement under the Significant Landscape Overlay Schedule 2. This is stated in clause 42.03-2 of the Nillumbik Planning Scheme, which requires a permit to be obtained for the construction of a building or to carry out works, including the pruning or lopping of the trunk of a native tree, unless a schedule to the overlay specifically states that a permit is not required. Therefore, if the schedule to the overlay does not specifically state that a permit is not required for pruning of a native tree, then a permit must be obtained.

        The relevant clause is:

        VC224 Permit requirement
        A permit is required to:
        - Construct a building or to carry out works. This does not apply:
        - If a schedule to this overlay specifically states that a permit is not required.
        - To the conduct of agricultural activities including ploughing and fencing (but not the construction of dams) unless a specific requirement for that activity is specified in a schedule to this overlay.
        - Construct a fence if specified in the schedule to this overlay.
        - Remove, destroy or lop any vegetation specified in a schedule to this overlay. This does not apply:
        - Lopping or pruning native vegetation, for maintenance only, provided no more than 1/3 of the foliage of each individual plant is lopped or pruned. Lopping and pruning for maintenance.
        - Native vegetation that is to be removed, destroyed or lopped to the minimum extent necessary by the holder of an exploration, mining, prospecting, or retention license issued under the Mineral Resources (Sustainable Development) Act 1990: Mineral exploration and extraction that is low impact exploration within the meaning of Schedule 4A of the Mineral Resources (Sustainable Development) Act 1990; or in accordance with a work plan approved under Part 3 of the Mineral Resources (Sustainable Development) Act 1990.

        Now answer the following question using the same format as above:
        Relevant documents:
        {summaries}

        Question: {question}
        Helpful answer in markdown or latex where equations are included:"""

        # Create PromptTemplate instances
        self.CONDENSE_QUESTION_PROMPT = PromptTemplate(
            input_variables=["chat_history", "question"], 
            template=self.CONDENSE_PROMPT
        )

        self.QA_PROMPT_TEMPLATE = PromptTemplate(
            input_variables=["summaries", "question"], 
            template=self.QA_PROMPT
        )

        # initialise llm instances and chains
        self.gpt3 = ChatOpenAI(temperature=0, model_name = 'gpt-3.5-turbo')
        self.gpt4 = ChatOpenAI(temperature=0, model_name = 'gpt-4')
        self.question_generator = LLMChain(llm=self.gpt3, prompt=self.CONDENSE_QUESTION_PROMPT)
        self.doc_chain = load_qa_with_sources_chain(self.gpt3, chain_type="stuff", prompt=self.QA_PROMPT_TEMPLATE)

        # declare main retrieval chain
        self.chain = ConversationalRetrievalChain(
            retriever=self.vectordb.as_retriever(),
            question_generator=self.question_generator,
            combine_docs_chain=self.doc_chain,
            return_source_documents = True
        )

    def non_db_response(self,query, chat_history):
        SYSTEM_MESSAGE = """You will act as an AI Assistant expert in the Nillumbik Planning Scheme that I am having a conversation with. You have the expertise of a lawer with 10 years of experience in local town planning regulation for Nillumbik in Melbourne, Australia. You will provide answers and guidance from your extensive knowledge and will always provide answer when requested with exact reference and outline from the source documents. Your response should be detailed and use as much of the relevant documents as possible. Your goal is to provide the best possible guidance and support to help me with my queries and problems related to your expertise.  

            Example:      
            
            Question: Does pruning of a native tree trigger a permit requirement under the Significant landscape overlay schedule 2?
            
            Response:
            Yes, pruning of a native tree does trigger a permit requirement under the Significant Landscape Overlay Schedule 2. This is stated in clause 42.03-2 of the Nillumbik Planning Scheme, which requires a permit to be obtained for the construction of a building or to carry out works, including the pruning or lopping of the trunk of a native tree, unless a schedule to the overlay specifically states that a permit is not required. Therefore, if the schedule to the overlay does not specifically state that a permit is not required for pruning of a native tree, then a permit must be obtained.

            The relevant clause is:

            VC224 Permit requirement
            A permit is required to:
            - Construct a building or to carry out works. This does not apply:
            - If a schedule to this overlay specifically states that a permit is not required.
            - To the conduct of agricultural activities including ploughing and fencing (but not the construction of dams) unless a specific requirement for that activity is specified in a schedule to this overlay.
            - Construct a fence if specified in the schedule to this overlay.
            - Remove, destroy or lop any vegetation specified in a schedule to this overlay. This does not apply:
            - Lopping or pruning native vegetation, for maintenance only, provided no more than 1/3 of the foliage of each individual plant is lopped or pruned. Lopping and pruning for maintenance.
            - Native vegetation that is to be removed, destroyed or lopped to the minimum extent necessary by the holder of an exploration, mining, prospecting, or retention license issued under the Mineral Resources (Sustainable Development) Act 1990: Mineral exploration and extraction that is low impact exploration within the meaning of Schedule 4A of the Mineral Resources (Sustainable Development) Act 1990; or in accordance with a work plan approved under Part 3 of the Mineral Resources (Sustainable Development) Act 1990.
        """
        
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": SYSTEM_MESSAGE},
                {"role": "user", "content": f"Given this previous conversation: {chat_history} and in the context of the Nillumbik Planning Scheme: {query}"},
            ],
        )
        return response["choices"][0]["message"]["content"]


    def summarise(self, fist_response, second_response, query):
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You will be provided with two responses to a question concurrently. Your task is to merge these responses into a more comprehensive and thorough one. Begin with the details from the second response, then interweave the details from the first response without leaving out any information. Your aim is to generate a richer, more complete answer than either individual response could offer. Since it relates to legal content, all numbers and exact values must be inlcluded in the reponse. If you encounter contradictions between the two responses, please present both viewpoints to provide a balanced perspective. Remember, absolutely every detail is valuable and needs to be included in the final output.  Respond using markdown."},
                {"role": "user", "content": f"Response 1: {fist_response} Response 2: {second_response} Question: {query}"},
            ],
        )
        return response["choices"][0]["message"]["content"]

    def chat_bot_response(self, message, chat_history):
        # chat_history.append(message)
        response1 = self.chain({"question": message, "chat_history": chat_history})
        response2 = self.non_db_response(message, chat_history)
        summary = self.summarise(response1['answer'],response2, message)
        return summary, response1['source_documents']
    
    ### Multithreaded version###
    def chat_bot_response_simult(self, message, chat_history):
        with concurrent.futures.ThreadPoolExecutor() as executor:
            future1 = executor.submit(self.chain, {"question": message, "chat_history": chat_history})
            future2 = executor.submit(self.non_db_response, message, chat_history)
            response1 = future1.result()
            response2 = future2.result()

        summary = self.summarise(response1['answer'], response2, message)
        return summary, response1['source_documents']

In [7]:
chat = ChatBot()

In [ ]:

response = chat.chat_bot_response('What are the regulations on landscaping or tree removal  in the Nillumbik Planning Scheme??', [])
response

In [8]:
response2 = chat.chat_bot_response_simult('What are the regulations on landscaping or tree removal  in the Nillumbik Planning Scheme??', [])
response2

("The Nillumbik Planning Scheme contains several sections and clauses that govern landscaping and tree removal. Key provisions and permit requirements can be found in the following sections:\n\n1. Vegetation Protection Overlay (VPO): Found under Clause 42.02, this overlay aims to protect and conserve native vegetation throughout the municipality. Permit requirements and exemptions for removing, lopping, or destroying vegetation are specified.\n\n2. Significant Landscape Overlays (SLO): Clause 42.03 addresses areas with significant landscape values. Schedules 2, 3, and 4 (SLO2, SLO3, SLO4) focus on significant trees and vegetation. These schedules outline permit requirements and exemptions for certain types of tree and vegetation removal, as well as pruning or lopping. Key requirements and exemptions are as follows:\n\n   - SLO2: A permit is required to remove, destroy, or lop any native vegetation, unless it's for maintenance only and no more than 1/3 of the foliage of each individual 

In [3]:
response

('Under the Nillumbik Planning Scheme, there are various clauses, schedules, and overlays that regulate landscaping and tree removal, including the Vegetation Protection Overlay (VPO), Significant Landscape Overlay (SLO), Environmental Significance Overlay (ESO), and other clauses such as Clause 52.17 - Native Vegetation and Clause 52.12 - Bushfire Protection: Exemptions. These aim to protect and enhance the natural environment and landscape values of the region.\n\n1. **Vegetation Protection Overlay (VPO) - Clause 42.02**: The VPO identifies significant vegetation and controls the removal, destruction, or lopping of this vegetation. A permit is required to remove, destroy, or lop any vegetation specified in a schedule to the overlay, subject to some exemptions such as removing vegetation listed as a noxious weed, for emergency access, or as stated in a specific schedule (Schedule 1 to 6).\n\n2. **Significant Landscape Overlay (SLO) - Clause 42.03**: The SLO aims to conserve and enhanc

In [27]:
response

{'question': 'What are the regulations on landscaping or tree removal  in the Nillumbik Planning Scheme??',
 'chat_history': [],
 'answer': 'The Nillumbik Planning Scheme has several regulations on landscaping and tree removal, depending on the specific overlay schedule that applies to the area in question. \n\nUnder the Significant Landscape Overlay Schedule 2, which applies to bush and semi-bush residential areas, a permit is required to remove, destroy, or lop any native vegetation, including substantial trees, unless the lopping is for maintenance only and no more than 1/3 of the foliage of each individual plant is lopped or pruned. A permit is also required to construct a front fence. However, a permit is not required to construct a building or carry out works if the building or works are at least 5 meters from the base of any substantial tree.\n\nUnder the Significant Landscape Overlay Schedule 3, which applies to areas with a bush garden character, a permit is required to remove

In [21]:
## For testing ###
def non_db_response(query, chat_history):
    SYSTEM_MESSAGE = """You will act as an AI Assistant expert in the Nillumbik Planning Scheme that I am having a conversation with. You have the expertise of a lawer with 10 years of experience in local town planning regulation for Nillumbik in Melbourne, Australia. You will provide answers and guidance from your extensive knowledge and will always provide answer when requested with exact reference and outline from the source documents. Your response should be detailed and use as much of the relevant documents as possible. Your goal is to provide the best possible guidance and support to help me with my queries and problems related to your expertise.  

        Example:      
        
        Question: Does pruning of a native tree trigger a permit requirement under the Significant landscape overlay schedule 2?
        
        Response:
        Yes, pruning of a native tree does trigger a permit requirement under the Significant Landscape Overlay Schedule 2. This is stated in clause 42.03-2 of the Nillumbik Planning Scheme, which requires a permit to be obtained for the construction of a building or to carry out works, including the pruning or lopping of the trunk of a native tree, unless a schedule to the overlay specifically states that a permit is not required. Therefore, if the schedule to the overlay does not specifically state that a permit is not required for pruning of a native tree, then a permit must be obtained.

        The relevant clause is:

        VC224 Permit requirement
        A permit is required to:
        - Construct a building or to carry out works. This does not apply:
        - If a schedule to this overlay specifically states that a permit is not required.
        - To the conduct of agricultural activities including ploughing and fencing (but not the construction of dams) unless a specific requirement for that activity is specified in a schedule to this overlay.
        - Construct a fence if specified in the schedule to this overlay.
        - Remove, destroy or lop any vegetation specified in a schedule to this overlay. This does not apply:
        - Lopping or pruning native vegetation, for maintenance only, provided no more than 1/3 of the foliage of each individual plant is lopped or pruned. Lopping and pruning for maintenance.
        - Native vegetation that is to be removed, destroyed or lopped to the minimum extent necessary by the holder of an exploration, mining, prospecting, or retention license issued under the Mineral Resources (Sustainable Development) Act 1990: Mineral exploration and extraction that is low impact exploration within the meaning of Schedule 4A of the Mineral Resources (Sustainable Development) Act 1990; or in accordance with a work plan approved under Part 3 of the Mineral Resources (Sustainable Development) Act 1990.
    """
    
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": SYSTEM_MESSAGE},
            {"role": "user", "content": f"Given this previous conversation: {chat_history} and in the context of the Nillumbik Planning Scheme: {query}"},
        ],
    )
    return response["choices"][0]["message"]["content"]



In [22]:
query ='What are the regulations on landscaping or tree removal  in the Nillumbik Planning Scheme?'
response2 = non_db_response(query, [])
response2

"In the Nillumbik Planning Scheme, there are multiple regulations concerning landscaping and tree removal, which are mainly found in Clause 42.03 Significant Landscape Overlay and Clause 52.17 Native Vegetation. The specific requirements and exemptions are subject to the particular schedule and zoning of your property. Let's outline the key regulations:\n\n1. Clause 42.03 Significant Landscape Overlay (SLO):\nThis clause provides an overlay designed to protect significant landscapes and conserve native trees. Depending on the schedule and zoning of your property, you may be subject to different requirements and exemptions. For example, Schedule 2 to the SLO (SLO2) focuses on tree conservation, which can trigger a permit requirement for removing, destroying, or lopping native vegetation, including pruning native trees.\n\n2. Clause 52.17 Native Vegetation:\nThis clause deals with the removal, destruction, or lopping of native vegetation, aiming to achieve a net gain in the quality and e

In [26]:
def summarise(fist_response, second_response, query):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You will be provided with two responses to a question concurrently. Your task is to merge these responses into a more comprehensive and thorough one. Begin with the details from the second response, then interweave the details from the first response without leaving out any information. Your aim is to generate a richer, more complete answer than either individual response could offer. Since it relates to legal content, all numbers and exact values must be inlcluded in the reponse. If you encounter contradictions between the two responses, please present both viewpoints to provide a balanced perspective. Remember, absolutely every detail is valuable and needs to be included in the final output.  Respond using markdown."},
            {"role": "user", "content": f"Response 1: {fist_response} Response 2: {second_response} Question: {query}"},
        ],
    )
    return response["choices"][0]["message"]["content"]

In [24]:
summary = summarise(response['answer'],response2, query)

In [25]:
summary

"In the Nillumbik Planning Scheme, there are multiple regulations concerning landscaping and tree removal, which are mainly found in Clause 42.03 Significant Landscape Overlay (SLO) and Clause 52.17 Native Vegetation. There are also Vegetation Protection Overlays (VPO) that may apply to some properties. The specific requirements and exemptions are subject to the particular schedule and zoning of your property. Let's outline the key regulations:\n\n1. **Clause 42.03 Significant Landscape Overlay (SLO)**: This clause provides an overlay designed to protect significant landscapes and conserve native trees. Depending on the schedule and zoning of your property, you may be subject to different requirements and exemptions.\n\n   - **SLO Schedule 2 (SLO2)**: Applies to bush and semi-bush residential areas. A permit is required to remove, destroy, or lop any native vegetation, including substantial trees, unless the lopping is for maintenance only and no more than 1/3 of the foliage of each in